In [7]:
import os, sys, glob, time

import numpy as np
import pandas as pd

import math

# Parallel proces
from myparproc.ParProc import ParProc
from myparproc.ProgressCheck import ProgressCheck

# Linear algebra
import myutil.myutil as util

# Plotting tools
import matplotlib.pyplot as plt

import rospy, rosbag

# For logging to csv
import csv

from livox_ros_driver.msg import CustomMsg as lvCloud
from livox_ros_driver.msg import CustomPoint as lvPoint

# For kd tree search
# import pcl, pcl_ros, pcl_msgs

# For load/unloading pcd/ply
import pypcd
from pypcd import pypcd

# For loading/unloading pcd/ply
# from pyntcloud import PyntCloud

# Some processing or plotting
import open3d as o3d

from ceva import Ceva

# Lidar topic
lidar_topic = '/livox/lidar'

# Location of the dataset
# bag_file = '/media/tmn/mySataSSD1/DATASETS/MCDVIRAL/PublishedSequencesUnzipped/ntu_day_10/ntu_day_10_mid70.bag'
bag_file = '/media/tmn/mySataSSD1/Experiments/ctgaupro/cloud_avia_mid.bag'

# Output location
output_dir = '/media/tmn/mySataSSD1/Experiments/ctgaupro/cloud_avia_mid'
os.makedirs(output_dir + lidar_topic.replace('/', '__'), exist_ok=True)

# Fields to export to ply
interested_fields = ['x', 'y', 'z', 'intensity', 't']


In [ ]:
# Export the ground truth
# Load the pointcloud in the bag
bag = rosbag.Bag(bag_file)

tf_topic = '/tf'
# Get total message count
tf_msg = bag.get_message_count(tf_topic)
print("tf msg: ", tf_msg)

lidar_gtr = {}
for topic, msg, t in bag.read_messages():
    if topic == tf_topic:
        for tf in msg.transforms:
            if tf.child_frame_id not in lidar_gtr.keys():
                lidar_gtr[tf.child_frame_id] = []
            p = tf.transform.translation
            q = tf.transform.rotation
            lidar_gtr[tf.child_frame_id].append((p.x, p.y, p.z, tf.header.stamp.to_sec(), tf.header.stamp.to_sec(), q.x, q.y, q.z, q.w))

# Create np array for tf
for lidar_name in lidar_gtr:
    cloudPose = np.array(lidar_gtr[lidar_name], dtype=[('x', '<f4'), ('y', '<f4'), ('z', '<f4'), ('intensity', '<f4'), ('t', '<f8'), ('qx', '<f4'), ('qy', '<f4'), ('qz', '<f4'), ('qw', '<f4')])
    pypcd.PointCloud.from_array(cloudPose).save_pcd(output_dir + '/' + lidar_name + '_gtr.pcd', compression='ascii')

In [ ]:

def lvCloudToArray(msg, cloud_idx):
    x = []
    y = []
    z = []
    i = []
    t = []
    
    for lvP in msg.points:
        x.append(lvP.x)
        y.append(lvP.y)
        z.append(lvP.z)
        i.append(lvP.reflectivity)
        t.append(lvP.offset_time)

    points = list(zip(x, y, z, i, t))
    points = np.array(points, dtype=[('x',         'f4'),
                                     ('y',         'f4'),
                                     ('z',         'f4'),
                                     ('intensity', 'f4'),
                                     ('t',         'i4'),
                                    ])

    return points

# Load the pointcloud in the bag
bag = rosbag.Bag(bag_file)

# Get total message count
total_msg = bag.get_message_count(lidar_topic)
count_max_digit = len(str(total_msg))
print("Total msg: ", total_msg, count_max_digit)

progress = 0
count = -1
for topic, msg, t in bag.read_messages():
    if topic == lidar_topic:
    
        # Increment counter
        count += 1

        # Timestamp
        timestamp = msg.header.stamp.to_sec()

        # Extract the point cloud from message
        lvpc_in = lvCloudToArray(msg, count)
        
        # Sort points by sampling time, then ring (lexsort uses the reverse order)
        lvpc_in = lvpc_in[np.argsort(lvpc_in, order='t')]
       
        # Make the file name
        lvpc_filename = output_dir + '/' + lidar_topic.replace('/', '__') + '/' + f'{timestamp:.9f}' + '.pcd'
        
        # Save the pointcloud
        pypcd.save_point_cloud_bin_compressed(pypcd.PointCloud.from_array(lvpc_in), lvpc_filename)
        
        # Report the progress
        if (math.floor(count/total_msg*100) != progress or count == total_msg - 1):
            progress = math.floor(count/total_msg*100)
            print(f'Export {progress:3d}%')
        # print(f'Seq: {seq}. Cloud {count}. SweepTime: {sweepTime}')

# Need to close the log file to get the text out
bag.close()